Generation de data_tr_lemm.csv
- Prend data_tr en entrée (= traduction de data.csv)

In [1]:
import os

# Désactiver le GPU en définissant CUDA_VISIBLE_DEVICES à un vide#
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import importlib

import RktnChallenge.RktnModel
importlib.reload(RktnChallenge.RktnModel)
from RktnChallenge.RktnModel import ModelTrainer


from RktnChallenge.preprocessing.tokenizeString import tokenizeString
from RktnChallenge.preprocessing.filterStopWords import filterStopWords
from RktnChallenge.preprocessing.mergeFeatures import mergeFeatures
from RktnChallenge.preprocessing.mostOccur import mostOccur
from RktnChallenge.preprocessing.Dropper import Dropper
from RktnChallenge.preprocessing.TokenListToString import TokenListToString



import RktnChallenge.preprocessing.filterChar
importlib.reload(RktnChallenge.preprocessing.filterChar)
from RktnChallenge.preprocessing.filterChar import filterChar

import RktnChallenge.preprocessing.lemmatize
importlib.reload(RktnChallenge.preprocessing.lemmatize)
from RktnChallenge.preprocessing.lemmatize import lemmatize

import RktnChallenge.preprocessing.regularExprSub
importlib.reload(RktnChallenge.preprocessing.regularExprSub)
from RktnChallenge.preprocessing.regularExprSub import regularExprSub

2023-09-08 13:11:45.571940: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-08 13:11:46.138412: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-08 13:11:47.644133: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-08 13:11:54.303668: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-08 1

In [2]:
RktnModel = ModelTrainer("data_tr.csv")
nom_sauvegarde = "data_tr_lemm.csv"

In [14]:
addendum = ['a','à','<p>','<b>','<div>','<em>','<br>','gt',
            'h','b','k','a','c','d','e','ex','Ndeg']

filtr = ['+','-','/','(',')',':',"'",'"',':','@''!','|','#','<','>','?',
         '1','2','3','4','5','6','7','8','9','0']

#on tokenise et retire les stopwords designation

desi_filter = filterChar("designation","designation", filtr)

desi_tokenize= tokenizeString("designation_tkn","designation")
desi_stopwords = filterStopWords("designation_tkn", "designation_tkn"
                                 ,lang=["english","french"],addendum=addendum)

desi_lemmatize = lemmatize("designation_tkn","designation_tkn")

#on tokenise et retire les stopwords description
desc_filter = filterChar("description","description", filtr)
desc_tokenize= tokenizeString("description_tkn","description")
desc_stopwords = filterStopWords("description_tkn", "description_tkn"
                                 ,lang=["english","french"],addendum=addendum)

desc_lemmatize = lemmatize("description_tkn","description_tkn")
merge_desi_desc = mergeFeatures("merged","designation_tkn","description_tkn")
desc_mostOccur= mostOccur("merged_desi_desc","merged",1000,1000)



toDrop = ["designation_tkn","description_tkn","merged"]
cleanDropper = Dropper(column_to_drop = toDrop)
toString = TokenListToString("merged_desi_desc","merged_desi_desc")



RktnModel.initPreprocess()
RktnModel.addPreprocessStep("clean designation",desi_filter) # filtre tous les caracteres et mots incorrects
RktnModel.addPreprocessStep("tokenize designation",desi_tokenize) # transformation en token
RktnModel.addPreprocessStep("filter designation",desi_stopwords)  # filtrage des stopwords
#RktnModel.addPreprocessStep("lemmitize designation",desi_lemmatize)  # Lemmatisation designation

RktnModel.addPreprocessStep("clean description",desc_filter) # filtre tous les caracteres et mots incorrects
RktnModel.addPreprocessStep("tokenize description",desc_tokenize) # transformation en token
RktnModel.addPreprocessStep("filter description",desc_stopwords)  # filtrage des stopwords
RktnModel.addPreprocessStep("lemmitize description",desc_lemmatize)  # Lemmatisation
RktnModel.addPreprocessStep("merge designation and desc",merge_desi_desc) # merge desi&desc en un seul champ
RktnModel.addPreprocessStep("desc reduce",desc_mostOccur)         # reduction taille desc
RktnModel.addPreprocessStep("tostring",toString) # convertit en chaine de caracteres


#RktnModel.addPreprocessStep("cleaning",cleanDropper) # on clean les colonnes crées
RktnModel.preprocess()

removeNumbers = regularExprSub("merged_desi_desc","merged_desi_desc",filtr='\d+')
RktnModel.initPreprocess()
RktnModel.addPreprocessStep("removeNumbers",removeNumbers) # on enleve les nombres qui pourraient rester
RktnModel.preprocess()

RktnModel.saveCSV(nom_sauvegarde)

[nltk_data] Downloading package stopwords to /home/pec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import pandas as pd
RktnModel = ModelTrainer("data_tr_lemm.csv")
RktnModel.data['description_length'] = RktnModel.data['description'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
RktnModel.data['designation_length'] = RktnModel.data['designation'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)

min_val = RktnModel.data['description_length'].min()
max_val = RktnModel.data['description_length'].max()
RktnModel.data['description_length_normalized'] = (RktnModel.data['description_length'] - min_val) / (max_val - min_val)

min_val = RktnModel.data['designation_length'].min()
max_val = RktnModel.data['designation_length'].max()
RktnModel.data['designation_length_normalized'] = (RktnModel.data['designation_length'] - min_val) / (max_val - min_val)

RktnModel.saveCSV(nom_sauvegarde)


(83493, 27)
